### Starts by defining the problem

In [ ]:
from itertools import product
from random import random, randint, shuffle, seed
import numpy as np
from scipy import sparse
from functools import reduce
from math import ceil, exp
from tqdm.auto import tqdm
from tabulate import tabulate
from copy import copy


def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points*2654435761+num_sets+density)
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets-1), p] = True
    return np.array(sets.toarray())

In [ ]:
DENSITY = 0.3 # [.3,.7]

sets100 = make_set_covering_problem(100, 100, DENSITY)
sets1000 = make_set_covering_problem(1000, 1000, DENSITY)
sets5000 = make_set_covering_problem(5000, 5000, DENSITY)

In [82]:
PROBLEM_SIZE = 5000 # [100, 1_000, 5_000]
SET_NUMBER = PROBLEM_SIZE
SETS = sets5000 # [sets100, sets1000, sets5000]

In [ ]:
def random_hill_climbing(sets, max_steps=10_000, calling = 0):
    """Returns the number of calls to the fitness function""" 
    calling = 0  
    def fitness(sets,state):
        cost = sum(state)
        valid = np.sum(
            reduce(
            np.logical_or,
            [sets[i] for i, t in enumerate(state) if t],
            np.array([False for _ in range(PROBLEM_SIZE)]),
            )
        )
        return valid , -cost , valid == sets.shape[0]

    def tweak(state):
        new_state = copy(state)
        index = randint(0, PROBLEM_SIZE - 1)
        new_state[index] = not new_state[index]
        return new_state

    current_state = [False for _ in range(sets.shape[0])]
    for step in range(max_steps):
        new_state = tweak(current_state)
        fitness_current = fitness(sets,current_state)
        fitness_new = fitness(sets,new_state)
        calling += 2
        if fitness_new >= fitness_current:
            current_state = new_state
        if fitness_current[2]:
            return calling
    return calling

sets = make_set_covering_problem(PROBLEM_SIZE, SET_NUMBER, DENSITY)
max_steps = ceil(SET_NUMBER / 2)
calls = random_hill_climbing(sets , max_steps)
print(f"Set Size = {sets.shape[0]}, fitness calls = {calls}")

### Simulated Annealing
0.3 density
Set Size = 5000, mean fitness calls = 12, best fitness calls = 10
Set Size = 1000, mean fitness calls = 11, best fitness calls = 9
Set Size = 100, mean fitness calls = 9, best fitness calls = 7
0.7 density
Set Size = 100, mean fitness calls = 7, best fitness calls = 6
Set Size = 1000, mean fitness calls = 9, best fitness calls = 8
Set Size = 5000, mean fitness calls = 10, best fitness calls = 9


In [ ]:
def visualize_set():
    """Visualize the set covering problem"""
    import matplotlib.pyplot as plt
    plt.figure(figsize=(10, 10))
    plt.imshow(SETS, cmap="gray" , aspect = 'auto')
    plt.title(f"Set Covering Problem with {PROBLEM_SIZE} points and {SET_NUMBER} sets")
    plt.xlabel("Points")
    plt.ylabel("Sets")
    plt.xticks(np.arange(0, PROBLEM_SIZE, 100))
    plt.show()

visualize_set()

In [ ]:
def simulated_annealing(sets, max_steps=10_000, calling = 0,starting_temp = 100):
    """Returns the number of calls to the fitness function""" 
    calling = 0  
    temp = starting_temp

    def tweak_simulated_annealing(current_state : np.array, max_pop : int = 5) -> np.array: 
        """Returns a neighbor of the given state"""
        state = current_state.copy()
        
        to_change = randint(0, max_pop)
        for _ in range(to_change):
            index = randint(0, PROBLEM_SIZE - 1)
            state[index] = not state[index]
        
        return state

    def fitness(sets,state):
        cost = sum(state)
        #print(state.shape)
        valid = np.sum(
            reduce(
            np.logical_or,
            [sets[i] for i, t in enumerate(state) if t],
            np.array([False for _ in range(PROBLEM_SIZE)]),
            )
        )
        return valid , -cost , valid == sets.shape[0]

    current_state = np.array([False for _ in range(sets.shape[0])])
    fitness_current = None
    for step in range(max_steps):
        if fitness_current is None:
            fitness_current = fitness(sets,current_state)
            calling += 1
        temp = 0.9 * temp
        new_state = tweak_simulated_annealing(current_state)
        fitness_new = fitness(sets,new_state)
        calling += 1
        #print(f"Annealing : {exp(-(fitness_current[0]-fitness_new[0])/temp)} , temp : {temp} , fitness_current : {fitness_current[0]} , fitness_new : {fitness_new[0]}")
        if exp(-(fitness_current[0]-fitness_new[0])/temp) >= random():
            fitness_current = fitness_new
            current_state = new_state
        if fitness_current[2]:
            return calling , current_state
        
    return calling , current_state

calls = 0
best = 100
max_steps = ceil(PROBLEM_SIZE / 2)
for i in range(100):
    temp , state = simulated_annealing(SETS , max_steps)
    if temp < best:
        best = temp
    calls += temp

print(f"Set Size = {SETS.shape[0]}, mean fitness calls = {ceil(calls/100)}, best fitness calls = {best}")


### Evolutionary Annealing
0.7 density
Set Size = 5000, mean fitness calls = 46, best fitness calls = 10
Set Size = 1000, mean fitness calls = 35, best fitness calls = 10
Set Size = 100, mean fitness calls = 18, best fitness calls = 5
0.3 density
Set Size = 100, mean fitness calls = 35, best fitness calls = 10
Set Size = 1000, mean fitness calls = 40, best fitness calls = 20
Set Size = 5000, mean fitness calls = 43, best fitness calls = 25


In [83]:
def evolutionary_annealing(sets,lam, max_steps=10_000, calling = 0,starting_temp = 100):
    """Returns the number of calls to the fitness function""" 
    calling = 0  
    temp = starting_temp

    def tweak_simulated_annealing(current_state : np.array, max_pop : int = 5) -> np.array: 
        """Returns a neighbor of the given state"""
        state = current_state.copy()
        
        to_change = randint(0, max_pop)
        for _ in range(to_change):
            index = randint(0, PROBLEM_SIZE - 1)
            state[index] = not state[index]
        
        return state

    def fitness(sets,state):
        cost = sum(state)
        #print(state.shape)
        valid = np.sum(
            reduce(
            np.logical_or,
            [sets[i] for i, t in enumerate(state) if t],
            np.array([False for _ in range(PROBLEM_SIZE)]),
            )
        )
        return valid , -cost , valid == sets.shape[0]

    current_state = np.array([False for _ in range(sets.shape[0])])
    fitness_current = None
    for step in range(max_steps):
        temp = 0.9 * temp

        new_state = None
        best_fitness = None
        for _ in range(lam):
            temp_state = tweak_simulated_annealing(current_state)
            fitness_new = fitness(sets,temp_state)
            #print(f"{fitness_new[0]} , {fitness_new[1]} , {fitness_new[2]}")
            
            if best_fitness is None or exp(-(best_fitness[0]-fitness_new[0])/temp) > random():
                best_fitness = fitness_new
                new_state = temp_state
            calling += 1
        #print(f"Annealing : {exp(-(fitness_current[0]-fitness_new[0])/temp)} , temp : {temp} , fitness_current : {fitness_current[0]} , fitness_new : {fitness_new[0]}")
        current_state = new_state
        fitness_current = best_fitness
        if fitness_current[2]:
            return calling , current_state
        
    return calling , current_state

calls = 0
best = 100
max_steps = ceil(PROBLEM_SIZE / 2)
for i in range(100):
    temp , state = evolutionary_annealing(SETS , 5 , max_steps)
    if temp < best:
        best = temp
    calls += temp

print(f"Set Size = {SETS.shape[0]}, mean fitness calls = {ceil(calls/100)}, best fitness calls = {best}")


Set Size = 5000, mean fitness calls = 43, best fitness calls = 25


| Algorithm                | Density | Set Size | Mean Fitness Calls | Best Fitness Calls |
|--------------------------|---------|----------|---------------------|---------------------|
| (1,&lambda;)-SE          | 0.7     | 100      | 18                 | 5                 |
|                          |         | 1000     | 35                 | 10                |
|                          |         | 5000     | 46                 | 10                |
|                          | 0.3     | 100      | 35                 | 10                |
|                          |         | 1000     | 40                 | 20                |
|                          |         | 5000     | 43                 | 25                |
|      Annealing           | 0.3     | 100      | 9                  | 7                 |
|                          |         | 1000     | 11                 | 9                 |
|                          |         | 5000     | 12                 | 10                |
|                          | 0.7     | 100      | 7                  | 6                 |
|                          |         | 1000     | 9                  | 8                 |
|                          |         | 5000     | 10                 | 9                 |
